In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/MyDrive/Colab [2
Notebooks/tmdb_5000_credits.csv')

SyntaxError: unterminated string literal (detected at line 2) (<ipython-input-3-46e61d624897>, line 2)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
movies.head(5)

In [ ]:
movies.info()

In [ ]:
credits.head(5)

In [ ]:
credits.info()

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.shape

In [ ]:
movies = movies[['movie_id','title','overview','genres','cast','keywords','crew']]

In [ ]:
movies.head(5) #merged dataframe

In [ ]:
movies.isnull().sum() #looking for missing values

In [ ]:
movies.dropna(inplace=True) #dropping the 3 movies where 3 entries in overview is missing

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
import ast

In [ ]:
# Genre Extraction function : from raw data for the creation of tags

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies.head(5)

In [ ]:
#function for extracting top(first) 3 actors from the movie

def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter !=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies.head(5)

In [ ]:
#function to fetch the director of movie from the crew column
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head(5)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
# applying a transformation to remove spaces between words

movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower()) #lower casing all the alphabets in the tags column

In [ ]:
new_df['tags'][0] # all letters are now in lower-case

In [ ]:
new_df.head()

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
import nltk #for stemming process

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
#defining the stemming function
def stem(text):
    y=[]

    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem) #appplying stemming process on tags column

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Avatar')  #enter movies only which are in the dataset, otherwise it would result in error

In [ ]:
import pickle

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))
#pickle.dump(similarity,open('movies.pkl','wb'))
#contains the similarity measure between different movies based on the tags

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))


pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))



In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))